<a href="https://colab.research.google.com/github/jonbaer/googlecolab/blob/master/Launch_Conditional_Config_Sweeps_(%2B_multi_objective_optimization).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conditional Configuration of Hyperparameter Search

W&B Launch allows you to express more sophisticated hyperparameter search logic, using libraries like [Optuna](https://optuna.org/#key_features) along with Python definitions for search logic.   This allows you to find optimal hyperparameters more efficiently--and it's particularly powerful when paired with W&B's ability to push searches to powerful compute clusters.

In this tutorial, you'll:


* Define conditional search logic with one level of nesting
* Save that logic to W&B
* Define a sweep configuration using Optuna
* Launch a sweep applying that logic to find optimal hyperparameters.

From here, you'll be positioned to massively scale up the sophistication and scale of your hyperparameter sweeps to improve your models.

Before you start, **make sure you have a Launch queue up and an agent running** (or follow [this guide](https://colab.research.google.com/drive/1wX0OSVxZJDHRsZaOaOEDx-lLUrO1hHgP#scrollTo=jhm3qUUxk69o&forceEdit=true&sandboxMode=true) to create one).  This is what will execute the sweep you define.




# Get set up with W&B

In [ ]:
# Install W&B
!pip install wandb>=0.15.8 -qqq

In [ ]:
# Log in to your free W&B Account
import wandb
import inspect
import yaml

wandb.login()


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
PROJECT = "fmnist_job_tutorial"
ENTITY = "" ## Put in your entity
QUEUE = "tutorial-run-queue" ## Put in a Launch queue you've created and started

# 2) Define conditional hyperparameter search logic

Here we're doing just one level of nesting, making the batch size conditional on the `train_size` setting.  But you can make the search space arbitrarily complex using Python.

In [ ]:
def objective(trial):
    """Optuna objective function to create a config using pythonic search spaces.
​
    Does not actually train, but is logged to wandb an artifact and used in the
    Optuna sweeps on launch scheduler."""

    train_size = trial.suggest_categorical('train_size', ['small', 'medium', 'large'])

    if train_size == 'small':
        batch_size = trial.suggest_int('batch_size', 16, 64)
    elif train_size == 'medium':
        batch_size = trial.suggest_int('batch_size', 8, 32)
    else:
        batch_size = trial.suggest_int('batch_size', 2, 16)

    lr = trial.suggest_float('learning_rate', 0.000001, 1.0)

    sleep = trial.suggest_float('sleep', 0.3, 1.0)

    a = trial.suggest_float('a', 0, 10)

    print(f"{train_size=} {batch_size=} {lr=} {sleep=} {a=}")

    # !! don't actually train, return -1
    return -1

# 3) Test the conditional configuration function.

We'll import Optuna to test the function we just constructed before we save it to W&B.

  

In [ ]:
! pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.2 MB/s eta 0:00:00


In [ ]:
    import optuna

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=2)
    print(f"Best trial: {study.best_trial.value=} {study.best_params=}")

[I 2023-10-11 17:28:32,961] A new study created in memory with name: no-name-634f909f-5dd4-4fe1-93d5-43c01307d0f7
[I 2023-10-11 17:28:32,990] Trial 0 finished with value: -1.0 and parameters: {'train_size': 'small', 'batch_size': 21, 'learning_rate': 0.5094918743625853, 'sleep': 0.9445360913643575, 'a': 1.059722445186102}. Best is trial 0 with value: -1.0.
[I 2023-10-11 17:28:33,002] Trial 1 finished with value: -1.0 and parameters: {'train_size': 'large', 'batch_size': 2, 'learning_rate': 0.9278913345116473, 'sleep': 0.9043828540124754, 'a': 0.9869390001700007}. Best is trial 0 with value: -1.0.


train_size='small' batch_size=21 lr=0.5094918743625853 sleep=0.9445360913643575 a=1.059722445186102
train_size='large' batch_size=2 lr=0.9278913345116473 sleep=0.9043828540124754 a=0.9869390001700007
Best trial: study.best_trial.value=-1.0 study.best_params={'train_size': 'small', 'batch_size': 21, 'learning_rate': 0.5094918743625853, 'sleep': 0.9445360913643575, 'a': 1.059722445186102}


# 4) Save the configuration to W&B as an artifact.

Now let's save the conditional search logic to W&B as an artifact.

In [ ]:
ARTIFACT_FILENAME = "optuna_wandb.py"
ARTIFACT_NAME = "optuna-config"

"""write function to its own file"""
function_lines = inspect.getsource(objective)
with open(ARTIFACT_FILENAME, 'w') as f:
    f.write(function_lines)

"""create and log artifact to wandb"""
run = wandb.init(project=PROJECT, entity=ENTITY)
artifact = run.log_artifact(ARTIFACT_FILENAME, name=ARTIFACT_NAME, type='optuna')
run.finish()

wandb: Currently logged in as: tristan-spaulding. Use `wandb login --relogin` to force relogin


# 5) Define a sweep configuration

Now that we've logged the condition configuration artifact, we'll define a configuration for our sweep.

(To adapt this for a different job, substitute your job for the top-level `job` field; here we'll use an [FMNIST job](https://colab.research.google.com/drive/1wX0OSVxZJDHRsZaOaOEDx-lLUrO1hHgP#scrollTo=tVR5Zn90h0pP).)

We'll use a custom Optuna scheduler.

In [ ]:
config = {
    "metric": {"name": "epoch/val_loss", "goal": "minimize"},
    "run_cap": 4,
    "job": "wandb/jobs/FMNIST Training:latest",
    "scheduler": {
        "job": "wandb/sweep-jobs/job-optuna-sweep-scheduler:latest",
        "num_workers": 2,
        "settings": {
            "optuna_source": f"{ENTITY}/{PROJECT}/{artifact.wait().name}",
            "optuna_source_filename": ARTIFACT_FILENAME,
            # optional sampler args
            "pruner": {
                "type": "PercentilePruner",
                "args": {
                    "percentile": 0.25,
                    "n_startup_trials": 2,
                    "n_min_trials": 1,  # min epochs before prune
                }
            }
        }
    },
    # parameters are not needed when loading a conditional config from an artifact
    # "parameters": {
    #     'epochs': {'values': [5, 10, 15]},
    #     'lr': {'max': 0.1, 'min': 0.0001}
    # }
}

# write config to file
config_filename = "sweep-config.yaml"
yaml.dump(config, open(config_filename, "w"))

#6) Launch the sweep!

We can now launch the sweep and see its progress in W&B.


In [ ]:
! wandb launch-sweep sweep-config.yaml -e $ENTITY -p $PROJECT -q $QUEUE

wandb: WARNING Using a scheduler job for launch sweeps is *experimental* and may change without warning
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb: launch: Launching run into tristan-spaulding/fmnist_job_tutorial
wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. 'parameters' is a required property
wandb: Created sweep with ID: tx33s6tt
wandb: View sweep at: https://wandb.ai/tristan-spaulding/fmnist_job_tutorial/sweeps/tx33s6tt
wandb: Scheduler added to launch queue (tutorial-run-queue2)


🖥 Go take a look at your terminal running the launch agent to see the sweep runs running.

##7) (Bonus) Create a multi-objective sweep

Optuna supports optimizing over multiple objectives at once, so in this example we will define a hyperparameter sweep to minimize validation loss, while simultaneously maximizing the categorical accuracy.

In [ ]:
config = {
    "run_cap": 4,
    "job": "wandb/jobs/FMNIST Training:latest",
    "scheduler": {
        "job": "wandb/sweep-jobs/job-optuna-sweep-scheduler:latest",
        "num_workers": 2,
        "settings": {
            "metrics": [
              {"name": "epoch/val_loss", "goal": "minimize"},
              {"name": "epoch/val_sparse_categorical_accuracy", "goal": "maximize"}
            ]
        }
    },
    "parameters": {
        'epochs': {'values': [2, 5, 10, 15]},
        'lr': {'max': 0.1, 'min': 0.0001}
    }
}

# write config to file
config_filename = "sweep-config-multi.yaml"
yaml.dump(config, open(config_filename, "w"))

# and launch the sweep
! wandb launch-sweep sweep-config-multi.yaml -e $ENTITY -p $PROJECT -q $QUEUE

wandb: WARNING Using a scheduler job for launch sweeps is *experimental* and may change without warning
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb: launch: Launching run into tristan-spaulding/fmnist_job_tutorial
wandb: Created sweep with ID: k4jukhtr
wandb: View sweep at: https://wandb.ai/tristan-spaulding/fmnist_job_tutorial/sweeps/k4jukhtr
wandb: Scheduler added to launch queue (tutorial-run-queue2)
